In [1]:
import timm as timm
import torch as torch
!pip install timm

You should consider upgrading via the '/home/cailen/PycharmProjects/pythonProject/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install -q kaggle

You should consider upgrading via the '/home/cailen/PycharmProjects/pythonProject/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
! mkdir ~ /.kaggle

mkdir: cannot create directory ‘/home/cailen/.kaggle’: File exists


In [4]:
! cp kaggle.json ~ /.kaggle /

In [5]:
! chmod 600 ~ /.kaggle / kaggle.json

In [6]:
!kaggle datasets download -d zippyz / cats- and -dogs-breeds-classification-oxford-dataset

cats-and-dogs-breeds-classification-oxford-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!unzip cats- and -dogs-breeds-classification-oxford-dataset.zip -d./ dataset

Archive:  cats-and-dogs-breeds-classification-oxford-dataset.zip
replace ./dataset/annotations/annotations/._trimaps? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import glob
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import os
from sklearn.model_selection import train_test_split

**Data Preparation**

In [2]:
read_for = open("./dataset/annotations/annotations/list.txt")
sample_files = []
class_decode = {}
sample_labels = []
for txt_line in read_for:
    if txt_line[0] == '#':
        continue
    file_name, clsaa_id, cat_dog_id, sub_class_id = txt_line.split(' ')
    if cat_dog_id == '1':
        sample_files.append(os.path.join("./dataset/images/images/", file_name + '.jpg'))
        sample_labels.append(sub_class_id)
        class_decode[int(sub_class_id.strip("\n"))] = file_name.split("_")[0]
read_for.close()
sample_labels = [int(elem.strip("\n")) for elem in sample_labels]

In [3]:
class_decode

{1: 'Abyssinian',
 2: 'Bengal',
 3: 'Birman',
 4: 'Bombay',
 5: 'British',
 6: 'Egyptian',
 7: 'Maine',
 8: 'Persian',
 9: 'Ragdoll',
 10: 'Russian',
 11: 'Siamese',
 12: 'Sphynx'}

In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [5]:
class MyImageDataset(Dataset):

    def __init__(self, labels, list_IDs, transforms):
        self.labels = labels
        self.list_IDs = list_IDs
        self.transforms = transforms

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        img = Image.open(self.labels[i])
        img = img.convert('RGB')
        img = self.transforms(img)
        y = self.list_IDs[i]
        return img, y

In [6]:
def show_batch(dataloader, rows, columns):
    data = iter(dataloader)
    fig = plt.figure(figsize=(15, 12))

    imgs, _ = data.next()

    for i in range(rows * columns):
        npimg = imgs[i].numpy()
        fig.add_subplot(rows, columns, i + 1)
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        plt.axis('off')
    plt.show()

In [73]:
size = (224, 224)
val_transform=transforms.Compose([
                              transforms.Resize(size),
                              transforms.ToTensor(),
                              # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                              ])

# transform = val_transform

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(0.3),
    transforms.RandomAffine(30, (0.2, 0.2), (0.15, 0.15)),
    transforms.RandomPerspective(distortion_scale=0.4),
    # transforms.RandomInvert(0.3),
    transforms.Resize(size),
    # transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.4, hue=0.1),
    transforms.ToTensor()
])

# sample_ds = MyImageDataset(sample_files, sample_labels, transform)
# sample_dl = DataLoader(sample_ds,batch_size=8) # , shuffle=True

# show_batch(sample_dl, 2, 4)

In [74]:
x_train, x_test, y_train, y_test = train_test_split(sample_files, sample_labels, random_state=42)
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
# device = "cpu"
# torch.backends.cudnn.benchmark = True

# Parameters
params = {'batch_size': 32,
          'shuffle': True,
          'num_workers': 2}

In [75]:
train_set = MyImageDataset(x_train, y_train, transform)
train_gen = torch.utils.data.DataLoader(train_set, **params)

val_set = MyImageDataset(x_test, y_test, val_transform)
val_gen = torch.utils.data.DataLoader(val_set, **params)

**Model learning**

In [28]:
!pip install timm

In [102]:
import timm
from tqdm import tqdm
from torchvision.models import vgg19

model = torch.nn.Sequential(*(list(timm.create_model('efficientnet_b7', pretrained=True, num_classes=1000).children())[:-1]))
# model = vgg19(pretrained=True).classifier[:-1]

No pretrained weights exist for this model. Using random initialization.


In [94]:
def extract_features(model, max_epochs=1):
    train_f = []
    train_lbl = []
    val_f = []
    val_lbl = []
    with torch.no_grad():
        for epoch in range(max_epochs):
            #   # Training
            for local_batch, local_labels in tqdm(train_gen):
                #     # Transfer to GPU
                #     # local_batch, local_labels = local_batch.to(device), local_labels.to(device)

                # Model computations
                # print(model.extract_features(local_batch))
                train_f.append(model(local_batch))
                train_lbl.append(local_labels)

                # Validation
            for local_batch, local_labels in tqdm(val_gen):
                # Transfer to GPU
                # local_batch, local_labels = local_batch.to(device), local_labels.to(device)

                # Model computations
                val_f.append(model(local_batch))
                val_lbl.append(local_labels)

    # train_f = np.vstack(train_f)
    # train_lbl = np.vstack(train_lbl)
    # val_f = np.vstack(val_f)
    # val_lbl = np.vstack(val_lbl)
    return [train_f, train_lbl], [val_f, val_lbl]

In [95]:
tr, v = extract_features(model, max_epochs=1)

100%|██████████| 19/19 [01:18<00:00,  4.12s/it]


In [96]:
train_f = np.vstack([elem.numpy() for elem in tr[0][:-1]])
train_l = np.hstack([elem.numpy() for elem in tr[1][:-1]])
val_f = np.vstack([elem.numpy() for elem in v[0][:-1]])
val_l = np.hstack([elem.numpy() for elem in v[1][:-1]])

In [97]:
val_l

array([ 8,  5, 10,  7,  4,  7, 11,  9,  2,  8,  3,  5, 11,  3, 10,  5,  7,
       10,  2,  6,  1, 10,  2,  9,  1,  2,  5,  1, 10,  3, 12, 10,  9,  4,
       12,  9, 12,  3,  9,  7,  9,  4,  6, 12,  4,  6,  8,  7,  7,  5,  9,
        9,  3,  7,  6,  2,  1,  7,  8, 11,  2,  8, 11,  8,  6,  9,  1,  1,
        6, 11,  9,  2,  5,  5,  2, 11, 10,  7,  1,  6,  1,  6,  9, 11,  4,
        2, 11, 10,  2,  6,  3,  7, 12,  1, 12,  8,  6,  6,  9, 11,  8, 12,
        3, 10, 11,  1,  3,  3,  2,  1,  8,  5, 10, 11,  5,  1,  1,  8, 10,
        4, 10, 10,  4,  6,  6,  1,  5,  7,  9,  7, 11, 11,  5,  9,  7,  5,
        3,  5,  5,  1,  8,  7,  4,  8,  2,  8,  5,  8,  9,  2,  9, 10,  5,
        9, 10,  5,  9,  9,  4,  7,  8,  1,  1,  4,  6,  8,  4,  1,  7, 11,
       10,  2,  7,  8,  5, 10, 12, 12,  6,  6,  7,  1, 10,  8, 12, 11,  3,
        2,  8, 10, 11, 11,  9,  1, 11,  2,  3, 10,  6,  6,  5,  7, 11,  6,
        6,  3,  9,  9,  6,  9,  7,  4,  1,  2,  2, 11,  6, 11,  8, 12,  3,
       10,  3,  5,  5,  6

In [98]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42)
clf.fit(train_f, train_l)

ValueError: Found array with dim 4. LogisticRegression expected <= 2.

In [92]:
clf.score(val_f, val_l)

0.8385416666666666

In [81]:
from sklearn.metrics import roc_auc_score

In [82]:
roc_auc_score(val_l, clf.predict_proba(val_f), multi_class="ovo")

0.9876408678612406

In [85]:
!pip install catboost
from catboost import CatBoostClassifier

In [86]:
cat = CatBoostClassifier(iterations=20)

In [87]:
cat.fit(train_f, train_l)

Learning rate set to 0.5
0:	learn: 2.1066469	total: 1.46s	remaining: 27.7s
1:	learn: 1.7671514	total: 2.8s	remaining: 25.2s
2:	learn: 1.5427398	total: 4.13s	remaining: 23.4s
3:	learn: 1.4156864	total: 5.47s	remaining: 21.9s
4:	learn: 1.2935135	total: 6.82s	remaining: 20.5s
5:	learn: 1.1668341	total: 8.18s	remaining: 19.1s
6:	learn: 1.0683464	total: 9.52s	remaining: 17.7s
7:	learn: 0.9990021	total: 10.9s	remaining: 16.4s
8:	learn: 0.9544866	total: 12.3s	remaining: 15.1s
9:	learn: 0.8922399	total: 13.7s	remaining: 13.7s
10:	learn: 0.8322075	total: 15.1s	remaining: 12.3s
11:	learn: 0.7778203	total: 16.5s	remaining: 11s
12:	learn: 0.7188634	total: 17.9s	remaining: 9.66s
13:	learn: 0.6749639	total: 19.4s	remaining: 8.3s
14:	learn: 0.6442322	total: 20.8s	remaining: 6.94s
15:	learn: 0.6206255	total: 22.2s	remaining: 5.54s
16:	learn: 0.5961385	total: 23.6s	remaining: 4.16s
17:	learn: 0.5523126	total: 25s	remaining: 2.78s
18:	learn: 0.5200591	total: 26.5s	remaining: 1.39s
19:	learn: 0.4823891	t

In [88]:
roc_auc_score(val_l, cat.predict_proba(val_f), multi_class="ovo")

0.9605417374099577

In [89]:
cat.score(val_f, val_l)

0.6805555555555556